This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

cols_output = ['final_answer_found',
               'final_model_queries', 'final_num_suffixes_checked']

# Load the results + get generated suffixes

In [5]:
# dir_single_query = '/home/chansingh/mntv1/single_query_math_9_17' # single-digit
# dir_single_query = '/home/chansingh/mntv1/single_query_math_double_digit'
dir_single_query = '/home/chansingh/mntv1/single_query_math_long_suffs' # suffix results
r = analyze_utils.load_results_and_cache(dir_single_query, save_file='r_long.pkl', only_keep_scalar=False)
r_single_query = pd.read_pickle(os.path.join(dir_single_query, 'r_long.pkl'))
r_single_query = analyze_utils.postprocess_results(r_single_query)

# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_9_17' # single-digit
# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_double_digit'
dir_suffix_search = '/home/chansingh/mntv1/suffix_math_long_suffs'
r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r_long.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r_long.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search)

# r = r_single_query
r = pd.concat((r_single_query, r_suffix_search)).sort_values('checkpoint')
# r.to_pickle(f'../results/prompt_gen/results_suffix_math.pkl')

100%|██████████| 90/90 [00:06<00:00, 14.35it/s]


# Test generalization acc post-hoc

In [11]:
r[['suffix_str_added', 'final_answer_added']].values

array([[list(['', ' multiply']),
        'To compute the answer, take the input number and multiply 2'],
       [list(['']),
        'To compute the answer, take the input numbers and multiply'],
       [list(['']),
        'To compute the answer, take the input numbers and multiply'],
       [list(['']),
        'To compute the answer, take the input numbers and add'],
       [list(['']),
        'To compute the answer, take the input numbers and subtract'],
       [list(['']),
        'To compute the answer, take the input numbers and divide'],
       [list(['']),
        'To compute the answer, take the input numbers and subtract'],
       [list(['']),
        'To compute the answer, take the input numbers and subtract'],
       [list(['']),
        'To compute the answer, take the input numbers and multiply'],
       [list(['']),
        'To compute the answer, take the input numbers and subtract'],
       [list(['', ' output', ' value', ' result', ' answer', ' corresponding']),
  

In [16]:
args = r.iloc[0]

In [19]:
np.random.seed(args.seed)
(dset, dset_test), check_answer_func, descr = data.get_data(
            args, args.task_name, n_shots=args.n_shots, train_split_frac=args.train_split_frac)

In [20]:
dset.shape

(1000, 4)

In [ ]:
model = prompt_classification.create_model(args.checkpoint)

In [ ]:
loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix=f'{description} '
)